# Git Credential Helpers are Awesome!

Resources:
- https://git-scm.com/docs/git-credential
- https://www.theserverside.com/blog/Coffee-Talk-Java-News-Stories-and-Opinions/Where-system-global-and-local-Windows-Git-config-files-are-saved
- https://stackoverflow.com/questions/53189190/git-multiple-credential-helper
- https://docs.aws.amazon.com/codecommit/latest/userguide/setting-up-https-windows.html#setting-up-https-windows-credential-helper

Debugging Helpers:

`git -c 'credential.helper=/compute-helpers/code/github_credential_helper.py' fetch`

For subtrees
`git -c 'credential.helper=/compute-helpers/code/github_credential_helper.py' subtree pull --prefix starostka`

`> You must provide <repository> <ref>`

Git context:

Credential helpers associate credentials with contexts, defined by their URL.

Attributes:
- protocol
- host
- path
- username
- password
- url

## Credential helper for Deepnote

Default helper configuration in system scope
```
[credential "https://github.com"]
  useHttpPath = true
  helper = "/compute-helpers/code/github_credential_helper.py"
```

### Credential context for subtrees
With added subtrees we must adjust the input to the helper.
```
[credential "https://github.com/Starostka/starostka.git"]
  useHttpPath = true
  helper = "/compute-helpers/code/github_credential_helper.py" # here it needs proper args 
```

So what we'd like is to tell git that when the remote is a subtree the credential context is based on the home repository.
We can do that by modifying the url, since the credential context is based on that:
```
  url = https://github.com/Starostka/home.git
```

The remaining attributes is derived from the url by the helper:
```
    host = github.com
    path = Starostka/home.git
    protocol = https
```

In [ ]:
#!/usr/bin/env python3
"""
This script is used as a Git credential helper https://git-scm.com/docs/git-credential
in our user PODs.
We configure git to call this script every time the user wants to communicate with Github repository
over https.
In case Deepnote does not have token for the requested repository, this script returns
empty string, which signals that next authentication helper should be used.
"""

import sys
from os import environ
from typing import Optional

import requests


def parse_input(inp: str) -> dict:
    ret = {}
    for line in inp.splitlines():
        name, value = line.split("=")
        ret[name] = value
    return ret


def fetch_key(
    project_id: str, token_service_secret: str, owner: str, repo: str
) -> Optional[str]:
    token_service_url = environ.get("DEEPNOTE_GITHUB_TOKEN_SERVICE_URL")
    if token_service_url is None:
        return
    response = requests.get(
        token_service_url,
        params={"projectId": project_id, "owner": owner, "repo": repo},
        headers={"Authorization": f"Bearer {token_service_secret}"},
    )
    if response.ok:
        return response.text


def parse_repo_path(repo_path: str) -> (str, str):
    owner, repo = repo_path.split("/")
    if repo.endswith(".git"):
        repo = repo[: -len(".git")]
    return owner, repo


def generate_credentials(
    project_id: str, token_service_secret: str, parsed_input: dict
) -> Optional[dict]:
    if project_id is None or token_service_secret is None:
        return

    host = parsed_input.get("host")
    path = parsed_input.get("path")
    protocol = parsed_input.get("protocol")

    if (
        None in (host, path, protocol)
        or "" in (host, path, protocol)
        or parsed_input.get("host") != "github.com"
        or protocol != "https"
    ):
        return

    owner, repo = parse_repo_path(path)
    key = fetch_key(project_id, token_service_secret, owner, repo)

    if key is None:
        return
    return {**parsed_input, "username": "x-access-token", "password": key}


def main(output=sys.stdout):
    project_id = environ.get("DEEPNOTE_PROJECT_ID", None)
    token_service_secret = environ.get("DEEPNOTE_RUNTIME_UUID", None)

    parsed_input = parse_input(sys.stdin.read())

    credentials = generate_credentials(project_id, token_service_secret, parsed_input)
    if credentials is not None:
        credentials_string = "\n".join(
            map(lambda item: f"{item[0]}={item[1]}", credentials.items())
        )
        print(credentials_string, file=output)


if __name__ == "__main__":
    main()

## Credential helper for SageMaker

## Credential helper for Datalore

## Credential helper for NextJournal

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7393f88e-6674-4a74-8a6c-ce9dc44581ca' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>